# Branch-and-bound algorithms as polynomial-time approximation schemes

In this notebook, we show the impact of the branching rule and the node selection strategy on the performance of the branch-and-bound algorithm for the multidimensional knapsack problem and the unrelated job scheduling problem.

### How to use this notebook

1. First, run the files `main_multi_knapsack.py` and `main_unrelated_job_scheduling` to obtain the results. These will be `.csv` files in the `output` folder.
2. Then, change the values of `alpha` and/or `epsilon` and the value of `metric` to obtain some statistics on the impact of different strategies.

### Packages

In [18]:
%matplotlib notebook
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
import numpy as np

## Multiknapsack problem

In [19]:
df = pd.read_csv("output/results_multiknapsack_random_instances_FINAL.csv")

df.head()

,seed,n_knapsacks,n_items,alpha,branching_rule,node_selection,best_solution,best_bound,runtime,depth,number_of_left_turns,nodes_explored,terminate,number_of_nodes_for_optimality,optimal_solution,opt_gap
0,0,2,5,0.5,critical_element,greatest_upper_bound,33.0,34.714286,0.000221,1,0,2,True,2,33.0,0.000000
1,0,2,5,0.5,profit_per_weight_ratio,greatest_upper_bound,33.0,34.714286,0.000196,1,0,2,True,2,33.0,0.000000
2,0,2,5,0.5,kolasar_rule,greatest_upper_bound,29.0,34.714286,0.000200,1,0,2,True,2,33.0,0.121212
3,0,2,5,0.5,critical_element,depth_first,33.0,34.714286,0.000201,1,0,2,True,2,33.0,0.000000
4,0,2,5,0.5,profit_per_weight_ratio,depth_first,33.0,34.714286,0.000202,1,0,2,True,2,33.0,0.000000


Select the values of `alpha` and the metric you want to visualize.

In [20]:
alpha = 0.97
assert alpha in [0.5, 0.8, 0.95, 0.97], "Invalid alpha"

metric = "runtime"
assert metric in ["nodes_explored", "runtime", "depth", "number_of_left_turns", "number_of_nodes_for_optimality", "opt_gap"], "Invalid metric"

# Just for a nice writing of the legend
metric_on_the_y_axis = " ".join(metric.split("_")).capitalize()
y_scale = "log"
if metric == "opt_gap":
    metric_on_the_y_axis = "Gap"
    y_scale = "linear"
if metric == "depth":
    metric_on_the_y_axis = "Depth"
    y_scale = "linear"

node_selection_strategies = ["depth_first", "breadth_first", "greatest_upper_bound"]
variable_selection_strategies = ["critical_element", "profit_per_weight_ratio", "kolasar_rule"]

strategy_pairs = list(product(node_selection_strategies, variable_selection_strategies))
items_knapsack_list = [(5, 2), (10, 2), (10, 5), (50, 2), (50, 5), (50, 10), (50, 15), (100, 2), (100, 5), (100, 10), (100, 15)
                       ]

print(strategy_pairs)


[('depth_first', 'critical_element'), ('depth_first', 'profit_per_weight_ratio'), ('depth_first', 'kolasar_rule'), ('breadth_first', 'critical_element'), ('breadth_first', 'profit_per_weight_ratio'), ('breadth_first', 'kolasar_rule'), ('greatest_upper_bound', 'critical_element'), ('greatest_upper_bound', 'profit_per_weight_ratio'), ('greatest_upper_bound', 'kolasar_rule')]


In [21]:
colors = [('o', "blue"), ('o',"black"), ('o', "red"), ('^', "cyan"), ('^', "magenta"), ('^', "green"), ('s',"orange"), ('s', "purple"), ('s', "grey")]

In [22]:
plt.figure()

idx = 0 # Where to plot
for n_items, n_knapsacks in items_knapsack_list:
    df_n_items_n_knapsack = df[(df["alpha"] == alpha) & (df["n_items"] == n_items) & (df["n_knapsacks"] == n_knapsacks)]
    for i, (node_selection, variable_selection) in enumerate(strategy_pairs):
        df_filtered = df_n_items_n_knapsack[(df_n_items_n_knapsack["node_selection"] == node_selection) & (df_n_items_n_knapsack["branching_rule"] == variable_selection)]
        plt.plot(idx, np.mean(df_filtered[metric]), colors[i][0], label=f"{node_selection} - {variable_selection}", color=colors[i][1], alpha=0.5)
    idx += 1

plt.legend(['DFS - CE', 'DFS - PPW', 'DFS - K', 'BFS - CE', 'BFS - PPW', 'BFS - K', 'GUB - CE',  'GUB - PPW', 'GUB - K'], loc='upper left')
plt.xlabel("number of items, number of knapsacks")
plt.ylabel(metric_on_the_y_axis)
plt.yscale(y_scale)
if metric == "opt_gap":
    plt.ylim(-0.01, 0.2)
plt.xticks(range(len(items_knapsack_list)), [f"{n_items}, {n_knapsacks}" for n_items, n_knapsacks in items_knapsack_list], rotation=45)
plt.savefig("./output/multi_knapsack_{}_{}.pdf".format(alpha, metric), bbox_inches='tight')
plt.show()

<IPython.core.display.Javascript object>

Just a focus on the comparison among greatest upper bound, breadth-first and depth-first.

## Unrelated job scheudling problem


In [23]:
df = pd.read_csv("output/results_unrelated_job_scheduling_random_instances.csv")

df.head()

,seed,n_machines,n_jobs,epsilon,branching_rule,node_selection,rounding_rule,lower_bound,best_solution,runtime,depth,nodes_explored,terminate,number_of_nodes_for_optimality,optimal_solution,opt_gap
0,0,2,5,0.5,max_min_proc,lowest_lower_bound,best_matching,lin_relax,100,0.008396,0,2,True,2,100,1.421085e-16
1,0,2,5,0.5,max_min_proc,lowest_lower_bound,all_to_shortest,lin_relax,100,0.006974,0,2,True,2,100,1.421085e-16
2,0,2,5,0.5,max_min_proc,lowest_lower_bound,best_matching,bin_search,100,0.045399,0,2,True,2,100,1.421085e-16
3,0,2,5,0.5,max_min_proc,lowest_lower_bound,all_to_shortest,bin_search,100,0.038551,0,2,True,2,100,1.421085e-16
4,0,2,5,0.5,max_min_proc,depth_first,best_matching,lin_relax,100,0.004712,0,2,True,2,100,1.421085e-16


Select the values of `epsilon` and the metric you want to visualize.


In [24]:
epsilon = 0.01
assert epsilon in [0.5, 0.1, 0.05, 0.01], "Invalid epsilon"

metric = "runtime"
assert metric in ["nodes_explored", "runtime", "depth", "number_of_nodes_for_optimality", "opt_gap"], "Invalid metric"

# Just for a nice writing of the legend
metric_on_the_y_axis = " ".join(metric.split("_")).capitalize()
y_scale = "log"
if metric == "opt_gap":
    metric_on_the_y_axis = "Gap"
    y_scale = "linear"
if metric == "depth":
    metric_on_the_y_axis = "Depth"
    y_scale = "linear"

node_selection_strategy_list = ["lowest_lower_bound", "depth_first", "breadth_first"]
lower_bound_list = ["lin_relax", "bin_search"]
branching_rule_list = ["max_min_proc"]
rounding_rule_list = ["best_matching", "all_to_shortest"]

strategy_quartets = list(product(node_selection_strategy_list, lower_bound_list, branching_rule_list, rounding_rule_list))
jobs_machines_list = [(5, 2), (10, 2), (10, 5), (50, 2), (50, 5), (50, 10), (100, 2)]

print(strategy_quartets)
print(len(strategy_quartets))

[('lowest_lower_bound', 'lin_relax', 'max_min_proc', 'best_matching'), ('lowest_lower_bound', 'lin_relax', 'max_min_proc', 'all_to_shortest'), ('lowest_lower_bound', 'bin_search', 'max_min_proc', 'best_matching'), ('lowest_lower_bound', 'bin_search', 'max_min_proc', 'all_to_shortest'), ('depth_first', 'lin_relax', 'max_min_proc', 'best_matching'), ('depth_first', 'lin_relax', 'max_min_proc', 'all_to_shortest'), ('depth_first', 'bin_search', 'max_min_proc', 'best_matching'), ('depth_first', 'bin_search', 'max_min_proc', 'all_to_shortest'), ('breadth_first', 'lin_relax', 'max_min_proc', 'best_matching'), ('breadth_first', 'lin_relax', 'max_min_proc', 'all_to_shortest'), ('breadth_first', 'bin_search', 'max_min_proc', 'best_matching'), ('breadth_first', 'bin_search', 'max_min_proc', 'all_to_shortest')]
12


In [25]:
colors =  [("s", "blue"), ("s", "black"), ("s", "red"), ("s", "cyan"), ("o", "magenta"), ("o", "green"), ("o", "orange"), ("o", "purple"), ("^", "grey"), ("^", "brown"), ("^", "pink"),  ("^", "navy")]

In [26]:
plt.figure()

idx = 0 # Where to plot
for n_jobs, n_machines in jobs_machines_list:
    df_n_items_n_knapsack = df[(df["epsilon"] == epsilon) & (df["n_jobs"] == n_jobs) & (df["n_machines"] == n_machines)]
    for i, (node_selection, lower_bound, branching_rule, rounding_rule) in enumerate(strategy_quartets):
        df_filtered = df_n_items_n_knapsack[(df_n_items_n_knapsack["node_selection"] == node_selection) & (df_n_items_n_knapsack["branching_rule"] == branching_rule) & (df_n_items_n_knapsack["rounding_rule"] == rounding_rule) & (df_n_items_n_knapsack["lower_bound"] == lower_bound)]
        plt.plot(idx, np.mean(df_filtered[metric]), colors[i][0], color=colors[i][1], alpha=0.5)
    idx += 1

plt.xlabel("number of jobs, number of machines")
plt.ylabel(metric_on_the_y_axis)
if metric == "opt_gap":
    plt.ylim(-0.0001, 0.005)
plt.yscale(y_scale) # You may or may not need logarithmic scale
plt.xticks(range(len(jobs_machines_list)), [f"{n_items}, {n_knapsacks}" for n_items, n_knapsacks in jobs_machines_list], rotation=45)

# [('lowest_lower_bound', 'lin_relax', 'max_min_proc', 'best_matching'), ('lowest_lower_bound', 'lin_relax', 'max_min_proc', 'all_to_shortest'), ('lowest_lower_bound', 'bin_search', 'max_min_proc', 'best_matching'), ('lowest_lower_bound', 'bin_search', 'max_min_proc', 'all_to_shortest'), ('depth_first', 'lin_relax', 'max_min_proc', 'best_matching'), ('depth_first', 'lin_relax', 'max_min_proc', 'all_to_shortest'), ('depth_first', 'bin_search', 'max_min_proc', 'best_matching'), ('depth_first', 'bin_search', 'max_min_proc', 'all_to_shortest'), ('breadth_first', 'lin_relax', 'max_min_proc', 'best_matching'), ('breadth_first', 'lin_relax', 'max_min_proc', 'all_to_shortest'), ('breadth_first', 'bin_search', 'max_min_proc', 'best_matching'), ('breadth_first', 'bin_search', 'max_min_proc', 'all_to_shortest')]
plt.legend(['LLB - LR - MMP - BM', 'LLB - LR - MMP - AS', 'LLB - BS - MMP - BM', 'LLB - BS - MMP - AS', 'DF - LR - MMP - BM', 'DF - LR - MMP - AS', 'DF - BS - MMP - BM', 'DF - BS - MMP - AS', 'BF - LR - MMP - BM', 'BF - LR - MMP - AS', 'BF - BS - MMP - BM', 'BF - BS - MMP - AS'], loc='upper left', fontsize='small')

plt.savefig("./output/unrelated_job_scheduling_{}_{}.pdf".format(epsilon, metric), bbox_inches='tight')
plt.show()

<IPython.core.display.Javascript object>